In [48]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

In [88]:
# дф со всеми файлами
file_table_df = pd.read_pickle('data_media/file_table.pkl')

# фильтруем, только .json
json_df = file_table_df.copy().query('extension == ".json"')

# фильтруем, все кроме .json
not_json_df = file_table_df.copy().query('extension != ".json"')

In [89]:
# делаем написание всех имён с маленьких букв
json_df['name']     = json_df.name.apply(lambda x: x.lower())
not_json_df['name'] = not_json_df.name.apply(lambda x: x.lower())

In [90]:
# добавляем колонку с расшерением оригинального фалйла
json_df['extension_orig'] = json_df.name.apply(lambda x: Path(x).suffix)

# фильтруем пропуски ""
json_df = json_df.query('extension_orig != ""')

# добавляем колонку с именем файла без расшерений
json_df['media_name'] = json_df.name.apply(lambda x: Path(x).stem)

# добавляем колонку путём до папки с файлом 
json_df['parent_path'] = json_df.path.apply(lambda x: x.parent)
not_json_df['parent_path'] = not_json_df.path.apply(lambda x: x.parent)

In [104]:
not_json_df.head()

,name,extension,path,parent_path
3,dsc01856,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,G:\Google Photo_extr\Takeout\Google Фото\Photo...
5,dsc01857,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,G:\Google Photo_extr\Takeout\Google Фото\Photo...
7,dsc01858,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,G:\Google Photo_extr\Takeout\Google Фото\Photo...
9,dsc01859,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,G:\Google Photo_extr\Takeout\Google Фото\Photo...
11,dsc01860,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,G:\Google Photo_extr\Takeout\Google Фото\Photo...


In [102]:
json_df.head()

,name,extension,path,extension_orig,media_name,parent_path
4,dsc01856.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01856,G:\Google Photo_extr\Takeout\Google Фото\Photo...
6,dsc01857.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01857,G:\Google Photo_extr\Takeout\Google Фото\Photo...
8,dsc01858.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01858,G:\Google Photo_extr\Takeout\Google Фото\Photo...
10,dsc01859.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01859,G:\Google Photo_extr\Takeout\Google Фото\Photo...
12,dsc01860.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01860,G:\Google Photo_extr\Takeout\Google Фото\Photo...


In [115]:
media_json_df = (
    not_json_df.merge(json_df
                      , how='left'
                      , left_on=['name', 'parent_path', 'extension']
                      , right_on=['media_name', 'parent_path', 'extension_orig'])
    [['name_x', 'extension_x', 'path_x', 'name_y']]
    .rename(columns = {'name_x': 'media_name'
                       , 'extension_x': 'extension'
                       , 'path_x': 'path'
                       , 'name_y': 'json_name'})
)

In [118]:
media_no_json = media_json_df[media_json_df.json_name.isna()]

In [119]:
media_no_json.head()

,media_name,extension,path,json_name
28,dsc01887-измененный,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
35,dsc01893-измененный,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
51,dsc01911-измененный,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
1534,дача_01(1),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
1535,дача_01(2),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN


```
	media_name	extension	path	json_name
28	dsc01887-измененный	.jpg	G:\Google Photo_extr\Takeout\Google Фото\Photo...	NaN
35	dsc01893-измененный	.jpg	G:\Google Photo_extr\Takeout\Google Фото\Photo...	NaN
51	dsc01911-измененный	.jpg	G:\Google Photo_extr\Takeout\Google Фото\Photo...	NaN
1534	дача_01(1)	.jpg	G:\Google Photo_extr\Takeout\Google Фото\Photo...	NaN
1535	дача_01(2)	.jpg	G:\Google Photo_extr\Takeout\Google Фото\Photo...	NaN
```



In [120]:
%%time
for i, json_row in json_df.iterrows():
    for j, media_row in media_no_json.iterrows():
        if json_row['extension_orig'] == media_row['extension']and json_row['path'].parent == media_row['path'].parent:
            match = re.search(f"{json_row['name'][:-len(json_row['extension_orig'])]}", media_row['media_name'])
            if match:
                media_json_df['json_name'].loc[i] = json_row['name']
                continue

Wall time: 1h 58min 36s


In [86]:
json_df

,name,extension,path,extension_orig,orig_name
4,dsc01856.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01856
6,dsc01857.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01857
8,dsc01858.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01858
10,dsc01859.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01859
12,dsc01860.jpg,.json,G:\Google Photo_extr\Takeout\Google Фото\Photo...,.jpg,dsc01860


In [122]:
media_json_df[media_json_df.json_name.isna()]

,media_name,extension,path,json_name
28,dsc01887-измененный,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
35,dsc01893-измененный,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
51,dsc01911-измененный,.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
1534,дача_01(1),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
1535,дача_01(2),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Photo...,NaN
...,...,...,...,...
31103,dsc_0214(1),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN
31105,dsc_0215(1),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN
31109,dsc_0218(1),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN
31149,dsc_0257(1),.jpg,G:\Google Photo_extr\Takeout\Google Фото\Савел...,NaN
